In [1]:
from gnn_tracking.metrics.losses import binary_focal_loss
import torch
import time

In [2]:
n = int(1e9)


In [5]:
start = time.time()
for _ in range(30):
    inpt = torch.rand(size=(n,), device="cuda")
    binary_focal_loss(inpt=inpt, target=inpt, alpha=0.3, pos_weight=inpt, gamma=2.)
print(start - time.time())

[15:47:55] WARNING: Masking 9917/1000000000 as outliers in focal loss
[15:47:55] DEBUG: tensor([0.9684, 0.2078, 0.9617, 0.7449, 0.7366, 0.8370, 0.3795, 0.1392, 0.7383,
        0.1610], device='cuda:0')
[15:47:55] DEBUG: tensor([0.9684, 0.2078, 0.9617, 0.7449, 0.7366, 0.8370, 0.3795, 0.1392, 0.7383,
        0.1610], device='cuda:0')
[15:47:55] WARNING: Masking 9901/1000000000 as outliers in focal loss
[15:47:55] DEBUG: tensor([0.0371, 0.2699, 0.5786, 0.0656, 0.4074, 0.0130, 0.9415, 0.6172, 0.1557,
        0.4469], device='cuda:0')
[15:47:55] DEBUG: tensor([0.0371, 0.2699, 0.5786, 0.0656, 0.4074, 0.0130, 0.9415, 0.6172, 0.1557,
        0.4469], device='cuda:0')
[15:47:55] WARNING: Masking 10153/1000000000 as outliers in focal loss
[15:47:55] DEBUG: tensor([0.0345, 0.3698, 0.3155, 0.5507, 0.1668, 0.9732, 0.3922, 0.3342, 0.4241,
        0.3211], device='cuda:0')
[15:47:55] DEBUG: tensor([0.0345, 0.3698, 0.3155, 0.5507, 0.1668, 0.9732, 0.3922, 0.3342, 0.4241,
        0.3211], device='cuda:0

-7.624078035354614


In [4]:
from torch import Tensor as T
from torch.nn.functional import relu


In [29]:

@torch.jit.script
def _condensation_loss(
        *, beta: T, x: T, particle_id: T, mask: T, q_min: float, radius_threshold: float
) -> dict[str, T]:
    pids = torch.unique(particle_id[particle_id > 0])
    # n_nodes x n_pids
    pid_masks = particle_id[:, None] == pids[None, :]  # type: ignore

    q = torch.arctanh(beta) ** 2 + q_min
    alphas = torch.argmax(q[:, None] * pid_masks, dim=0)
    x_alphas = x[alphas].transpose(0, 1)
    q_alphas = q[alphas][None, None, :]

    diff = x[:, :, None] - x_alphas[None, :, :]
    norm_sq = torch.sum(diff**2, dim=1)

    # Attractive potential
    va = q[:, None] * pid_masks * (norm_sq * q_alphas).squeeze(dim=0)
    # Repulsive potential
    vr = (
            q[:, None]
            * (~pid_masks)
            * (
                    relu(radius_threshold - torch.sqrt(norm_sq + 1e-8)) * q_alphas
            ).squeeze(dim=0)
    )

    return {
        "attractive": torch.sum(torch.mean(va[mask], dim=0)),
        "repulsive": torch.sum(torch.mean(vr, dim=0)),
    }


In [31]:
_condensation_loss(beta=beta, x=x, particle_id=particle_id, q_min=0.5, radius_threshold=1.5, mask=mask)

{'attractive': tensor(15.0197), 'repulsive': tensor(1642.4094)}

In [32]:
a = time.time()
n = int(1e5)
beta = torch.rand(size=(n,))
x=torch.rand(size=(n,4))
particle_id = (100*torch.rand(size=(n,))).long()
mask = beta > 0
for _ in range(100):
    _condensation_loss(beta=beta, x=x, particle_id=particle_id, q_min=0.5, radius_threshold=1.5, mask=mask)
print(time.time() - a)

4.599055767059326


In [34]:
4.6/6.6

0.6969696969696969